# TemplateFlow

A resource for handling neuroimaging templates and atlases.

Components:

* The *TemplateFlow Archive*
* The *Python Client*
* The *TemplateFlow Registration Framework*

Check [www.templateflow.org](https://www.templateflow.org)

# The Python Client
Provides an easy interface to access the *Archive* by humans and programs.

## Easy to install

In [ ]:
!pip install templateflow --user

# Easy to use
The `api` submodule contains all the utilities

In [3]:
from templateflow import api

For instance, we can query the *Archive* to learn the list of templates currently available:

In [ ]:
api.templates()

Let's say we want to use the CerebrA parcellation recently published by the MNI group. The parcellation is defined on their `"MNI152NLin2009cSym"` template:

In [ ]:
api.get("MNI152NLin2009cSym", atlas="CerebrA")

## Lazy-loading
The first time we request a file, it gets downloaded to the `$TEMPLATEFLOW_HOME` folder, which by default points to `$HOME/.cache/templateflow`. The second time we run the same query, it's already fetched and available:

In [ ]:
api.get("MNI152NLin2009cSym", atlas="CerebrA")

## PyBIDS-like interface
The client just defines a new configuration object for PyBIDS, which is actually used in indexing and querying the Archive. For example, let's say we will need the anatomical reference of the CerebrA parcellation:

In [ ]:
api.get("MNI152NLin2009cSym", suffix="T1w")

# The *TemplateFlow Archive*

The *Archive* is a replicated repository (stored on the OSF and a public S3 bucket) managed and version-controlled with DataLad.

We are building a contribution workflow, so that users can easily upload their templates.

Currently available templates can be browsed with the [online browser](https://www.templateflow.org/archive/#browse-the-archive)

Accessing the Archive can be done with the Python client (recommended) or directly with Datalad (discouraged):

In [ ]:
!datalad install -r ///templateflow

In [ ]:
!ls templateflow/

# Use-case: extracting time-series from preprocessed data
The following use-case shows how to use the Finger/Foot/Lips motor task in DS000114, after preprocessing with fMRIPrep.

The full dataset after preprocessing has been uploaded to [G-Node](https://gin.g-node.org/oesteban/ds000114/). We will extract the time-series for subject `05`, session `test`, and task `fingerfootlips`. Therefore, we will work on [this file](), for which there's a symlink under `~/data/nipreps/templateflow/`.

In [15]:
from os import path as op
data_file = op.expanduser(
    "~/data/nipreps/templateflow/"
    "sub-05_ses-test_task-fingerfootlips_space-MNI152NLin6Asym_desc-preproc_bold.nii.gz"
)

We will use the Schaefer parcellation with 7 networks and 100 ROIs, at the lower resolution (``res-02``) of the reference template (which is FSL's MNI, or `MNI152NLin6Asym` for TemplateFlow).

In [18]:
api.get("MNI152NLin6Asym", atlas="Schaefer2018", resolution=2, desc="100Parcels7Networks")

PosixPath('/home/jovyan/.cache/templateflow/tpl-MNI152NLin6Asym/tpl-MNI152NLin6Asym_res-02_atlas-Schaefer2018_desc-100Parcels7Networks_dseg.nii.gz')